## Setup

#### Load the API key and relevant Python libaries.


In [3]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

openai.api_key = os.environ["OPENAI_API_KEY"]

client = openai.OpenAI()

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

## Prompt the model and get a completion


In [5]:
response = get_completion("What is the capital of France?")

In [6]:
print(response)

The capital of France is Paris.


## Helper function (chat format)


In [7]:
def get_completion_from_messages(
    messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500
):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
        max_tokens=max_tokens,  # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message.content

In [8]:
messages = [
    {
        "role": "system",
        "content": "You are an assistant who responds in the style of Dr Seuss.",
    },
    {"role": "user", "content": "write me a very short poem about a happy carrot"},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In a garden so bright, with soil so fine,
Lived a cheerful carrot, so sweet and so divine.
With leaves of green and orange so bright,
This happy carrot brought delight.
Dancing in the sun, growing tall and proud,
This jolly veggie cheered the crowd.
Oh, what joy this carrot brings,
With its happy heart that truly sings!


In [9]:
# length
messages = [
    {
        "role": "system",
        "content": "All your responses must be \
one sentence long.",
    },
    {"role": "user", "content": "write me a story about a happy carrot"},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Once there was a cheerful carrot named Charlie who always brightened up the vegetable patch with his contagious smile.


In [10]:
# combined
messages = [
    {
        "role": "system",
        "content": "You are an assistant who responds in the style of Dr Seuss. All your responses must be one sentence long.",
    },
    {"role": "user", "content": "write me a story about a happy carrot"},
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In a garden so bright, a happy carrot grew with all its might.


In [13]:
def get_completion_and_token_count(
    messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500
):

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message.content

    response.usage.total_tokens

    token_dict = {
        "prompt_tokens": response.usage.prompt_tokens,
        "completion_tokens": response.usage.completion_tokens,
        "total_tokens": response.usage.total_tokens,
    }

    return content, token_dict

In [14]:
messages = [
    {
        "role": "system",
        "content": "You are an assistant who responds in the style of Dr Seuss.",
    },
    {"role": "user", "content": "write me a very short poem about a happy carrot"},
]
response, token_dict = get_completion_and_token_count(messages)

In [15]:
print(response)

Oh, the happy carrot, so bright and so orange,
In the garden it grows, a joyful storage.
With a leafy green top and a crunchy bite,
It brings smiles to all, such a delightful sight!


In [16]:
print(token_dict)

{'prompt_tokens': 35, 'completion_tokens': 44, 'total_tokens': 79}
